In [1]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px # this is another plotting library for interactive plot

from sklearn.model_selection import train_test_split
from sklearn import metrics, manifold # we will use the metrics and manifold learning modules from scikit-learn
from pathlib import Path # to interact with file paths
from PIL import Image # to interact with images
from tqdm import tqdm # progress bar
from pprint import pprint # pretty print (useful for a more readable print of objects like lists or dictionaries)
from IPython.display import clear_output # to clear the output of the notebook

import torch
import torch.nn as nn
import torchvision
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
import os
import shutil


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
device = torch.device("cpu")

cuda


In [2]:
# CONTROL
num_channels = 1
SIZE = (128,128)
sign_ROI = [0,200,-200,640]
model_name = 'models/sign_classifier_small.pt'
onnx_lane_keeper_path = "models/sign_classifier_small.onnx"
max_load = 1_000

In [3]:
#load signs
signs_path = 'sign_imgs'
signs_names = ['park', 'closed_road', 'highway_exit', 'highway_enter', 'stop', 'roundabout', 'priority', 'cross_walk', 'one_way', 'no_sign']
signs_imgs = [cv.imread(os.path.join(signs_path, name + '.png'), cv.IMREAD_GRAYSCALE) for name in signs_names]
tot_signs = len(signs_imgs)

In [4]:
import cv2 as cv
import numpy as np

def load_and_augment_img(i, folder='training_imgs', sign_index=0, signs_imgs=signs_imgs):
    # img = cv.imread(os.path.join(folder, f'img_{i+1}.png'))
    img = np.zeros((SIZE[0], SIZE[1], num_channels), dtype=np.uint8)
    #convert to gray
    # img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    #crop in the sign ROI
    img = img[sign_ROI[0]:sign_ROI[1], sign_ROI[2]:sign_ROI[3]]

    #add initial noise to image
    std = 150
    std = np.random.randint(1, std)
    noisem = np.random.randint(0, std, img.shape, dtype=np.uint8)
    img = cv.subtract(img, noisem)
    noisep = np.random.randint(0, std, img.shape, dtype=np.uint8)
    img = cv.add(img, noisep)
    #add iniial blur
    std = 5
    std = np.random.randint(1, (std,std))

    #load sign
    sign = signs_imgs[sign_index]

    #perspective transform sign
    perspective_deformation = 100
    pts1 = np.float32([[0,0],[sign.shape[1],0],[sign.shape[1],sign.shape[0]],[0,sign.shape[0]]])
    pts2 = np.float32([[0,0],[sign.shape[1],0],[sign.shape[1],sign.shape[0]],[0,sign.shape[0]]])
    pts2 = pts2 + np.float32(np.random.randint(0,perspective_deformation,size=pts2.shape))
    # print(f'pts2 = \n{pts2}')
    new_size_x = int(np.max(pts2[:,0]) - np.min(pts2[:,0]))
    new_size_y = int(np.max(pts2[:,1]) - np.min(pts2[:,1]))
    M = cv.getPerspectiveTransform(pts1,pts2)
    sign = cv.warpPerspective(sign,M,(new_size_x,new_size_y))

    #resize sign keeping proportions
    img_sign_ratio = min(img.shape[0], img.shape[1]) / max(sign.shape[0], sign.shape[1])  
    scale_factor = np.random.uniform(.2, .8)
    scale_factor = scale_factor * img_sign_ratio
    sign = cv.resize(sign, (0,0), fx=scale_factor, fy=scale_factor)
    #match img shape
    sign_canvas = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    #get a random position for the sign
    sign_y = np.random.randint(0, img.shape[0] - sign.shape[0])
    sign_x = np.random.randint(0, img.shape[1] - sign.shape[1])
    #paste sign on canvas
    sign_canvas[sign_y:sign_y+sign.shape[0], sign_x:sign_x+sign.shape[1]] = sign
    #paste canvas on img
    img = np.where(sign_canvas > 0, sign_canvas, img) 

    #create random ellipses to simulate light from the sun
    light = np.zeros(img.shape, dtype=np.uint8)
    #add ellipses
    for j in range(2):
        cent = (np.random.randint(0, img.shape[0]), np.random.randint(0, img.shape[1]))
        axes_length = (np.random.randint(10, 50), np.random.randint(50, 300))
        angle = np.random.randint(0, 360)
        light = cv.ellipse(light, cent, axes_length, angle, 0, 360, 255, -1)
    #create an image of random white and black pixels
    light = cv.blur(light, (100,100))
    noise = np.random.randint(0, 2, size=img.shape, dtype=np.uint8)*255
    light = cv.subtract(light, noise)
    light = 5 * light
    #add light to the image
    img = cv.add(img, light)

    # #resize 
    # img = cv.resize(img, SIZE)

    # #reduce contrast
    # const = np.random.uniform(0.1,1.2)
    # if np.random.uniform() > 5:
    #     const = const*0.2
    # img = 127*(1-const) + img*const
    # img = img.astype(np.uint8)

    # #add noise 
    # std = 150
    # std = np.random.randint(1, std)
    # noisem = np.random.randint(0, std, img.shape, dtype=np.uint8)
    # img = cv.subtract(img, noisem)
    # noisep = np.random.randint(0, std, img.shape, dtype=np.uint8)
    # img = cv.add(img, noisep)
    # #blur 
    # img = cv.blur(img, (5,5))

    # #add random brightness
    # max_brightness = 50
    # brightness = np.random.randint(-max_brightness, max_brightness)
    # if brightness > 0:
    #     img = cv.add(img, brightness)
    # elif brightness < 0:
    #     img = cv.subtract(img, -brightness)
    
    # # invert color
    # if np.random.uniform(0, 1) > 0.6:
    #     img = cv.bitwise_not(img)

    return img


cv.namedWindow('img', cv.WINDOW_NORMAL)
# cv.setWindowProperty('img', cv.WND_PROP_FULLSCREEN, cv.WINDOW_FULLSCREEN)

sign_index = 0
for i in range(500):
    img = load_and_augment_img(i, sign_index=sign_index)
    cv.imshow('img', img)
    key = cv.waitKey(0)
    if key == ord('q') or key == 27:
        break
    sign_index = (sign_index + 1) % tot_signs
cv.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'